In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *
# Ensure the below library is installed on your cluster
from datetime import *

In [0]:
silver_path_rig="earth_data.silver.baker_rig_count"
rig_count_df=spark.read.table(silver_path_rig)


In [0]:
silver_path_earth="earth_data.silver.earthquake"
earthquake_count_df=spark.read.table(silver_path_earth)


In [0]:
earthquake_count_df.display()

In [0]:
silver_path_counties="earth_data.silver.us_counties"
us_counties_df=spark.read.table(silver_path_counties)
us_counties_df.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS earth_data.gold.agg_earthquake_rig_by_counties
AS
WITH rig_by_state AS (
   SELECT 
      state_or_province,
      county,
        SUM(RIG_COUNT) AS total_rigs,
        AVG(RIG_COUNT) AS avg_daily_rigs,
        MAX(RIG_COUNT) AS peak_rigs
    FROM earth_data.silver.baker_rig_count
    WHERE country ='UNITED STATES' and county IS NOT NULL
    GROUP BY state_or_province, county
),

earthquake_by_state AS (
  SELECT 
       county,
        COUNT(*) AS total_earthquakes,
        AVG(magnitude) AS avg_magnitude,
        MAX(magnitude) AS max_magnitude,
        COUNT_IF(magnitude>= 3.0) AS significant_events,
        AVG(sig) AS avg_significance
    FROM earth_data.silver.earthquake
    WHERE county IS NOT NULL
    GROUP BY county
),

combined_data AS (
    SELECT 
       r.county,
        r.state_or_province,       
        COALESCE(r.total_rigs, 0) AS total_rigs,
        COALESCE(e.total_earthquakes, 0) AS total_earthquakes,
        COALESCE(e.avg_magnitude, 0) AS avg_magnitude
    FROM rig_by_state r
    INNER JOIN earthquake_by_state e
        ON r.county = e.county

)

SELECT 
*
FROM combined_data
ORDER BY total_rigs DESC


